<a href="https://colab.research.google.com/github/muntazir02/maven/blob/main/muntazir_nlp_module_0_airline_tweet_sentiment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Assignement Submission by Muntazir Mehdi

## Twitter Sentiment Analysis

**Problem statement:** Airline industry had a very hard time post covid to sustain their business due to a long hault. It is very important for them to make sure they exceed customer expectations. The best way to evaluate performance is customer feedback. You are given a dataset of airline tweets from real customers.

A sentiment analysis job about the problems of each major U.S. airline. Twitter data was scraped from February of 2015 and contributors were asked to first classify positive, negative, and neutral tweets, followed by categorizing negative reasons (such as "late flight" or "rude service").

You will use the text column and sentiment column to create a classification model that classifies a given tweet into one of the 3 classes - positive, negative, neutral.

**Understanding the Dataset:**

Dataset contains many columns out of which below are most important ones-
1. airline_sentiment - defines the sentiment of the tweet 
2. negative_reason - reason for the negative feedback (if negative)
3. Text - tweet text content
4. tweet_location - location from which tweet was posted

You can use more columns in your model training if you want. 


**Steps to perform**
1. Load dataset - https://www.kaggle.com/datasets/crowdflower/twitter-airline-sentiment
2. Clean, preprocess data and EDA
3. Vectorise columns that contain text 
4. Run Classification model to classify - positive, negative or neutral
5. Evaluate model



## Steps to Download kaggle datasets using Kaggle Public API

1. Go to your account, Scroll to API section and Click Expire API Token to remove previous tokens

2. Click on Create New API Token - It will download kaggle.json file on your machine.

In [ ]:
!pip install -q kaggle

In [ ]:
from google.colab import files
files.upload()

{}

In [ ]:
! mkdir ~/.kaggle

! cp kaggle.json ~/.kaggle/

mkdir: cannot create directory ‘/root/.kaggle’: File exists
cp: cannot stat 'kaggle.json': No such file or directory


In [ ]:
! chmod 600 ~/.kaggle/kaggle.json

chmod: cannot access '/root/.kaggle/kaggle.json': No such file or directory


## Loading Data

In [ ]:
from google.colab import files
files.upload()
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/

Saving kaggle.json to kaggle.json


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!kaggle datasets download -d crowdflower/twitter-airline-sentiment
!unzip twitter-airline-sentiment.zip

twitter-airline-sentiment.zip: Skipping, found more recently modified local copy (use --force to force download)
Archive:  twitter-airline-sentiment.zip
replace Tweets.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace database.sqlite? [y]es, [n]o, [A]ll, [N]one, [r]ename: n


In [ ]:
import pandas as pd
df = pd.read_csv('Tweets.csv')
df.head(5)

# Basic Text Processing

In [ ]:
# Purpose - clean the text
import re
import spacy
import pandas as pd

nlp = spacy.load('en_core_web_sm')
data = df
def preprocess_text(text):
    text = re.sub(r'http\S+', '', text) #remove URLs
    text = re.sub(r'@[^\s]+', '', text) #remove mentions
    text = re.sub(r'[^\w\s]', '', text) #remove special characters
    text = re.sub(r'\n', ' ', text)  # remove newline characters
    text = re.sub(r'\s+', ' ', text)  # remove extra spaces
    text = text.lower() #lower case
    
    doc = nlp(text)
    words = [token.text for token in doc if not token.is_stop] #Remove stop words
    words = [token.text for token in doc if not token.is_punct] #remove punctuation marks
    text = ' '.join(words)    
    return text

data['text'] = data['text'].apply(preprocess_text)

# EDA

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# Countplot of the 'airline_sentiment' column from the 'data' dataframe
sns.countplot(x='airline_sentiment', data=data)
plt.show()  # Display the plot

# Countplot of the 'negativereason' column from the 'data' dataframe
sns.countplot(x='negativereason', data=data)
plt.xticks(rotation=90)  # Rotate x-axis labels by 90 degrees for better readability
plt.show()  # Display the plot


# Vectorize, Model & Test

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Initialize the TfidfVectorizer
vectorizer = TfidfVectorizer()

# Convert the 'text' column of the 'df' DataFrame to a TF-IDF matrix
X = vectorizer.fit_transform(df['text'])

from sklearn.preprocessing import LabelEncoder

# Initialize the LabelEncoder
le = LabelEncoder()

# Encode the 'airline_sentiment' column of the 'df' DataFrame
y = le.fit_transform(df['airline_sentiment'])

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize the Logistic Regression model
model = LogisticRegression()

# Train the model on the training data
model.fit(X_train, y_train)

from sklearn.metrics import classification_report

# Make predictions on the test data
predictions = model.predict(X_test)

# Print the classification report
print(classification_report(y_test, predictions, target_names=le.classes_))

              precision    recall  f1-score   support

    negative       0.83      0.94      0.88      1889
     neutral       0.67      0.54      0.60       580
    positive       0.83      0.58      0.68       459

    accuracy                           0.80      2928
   macro avg       0.78      0.69      0.72      2928
weighted avg       0.80      0.80      0.79      2928



/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
